# Build a Conditional GAN

### Goals
In this notebook, you're going to make a conditional GAN in order to generate hand-written images of digits, conditioned on the digit to be generated (the class vector). This will let you choose what digit you want to generate.

You'll then do some exploration of the generated images to visualize what the noise and class vectors mean.  

### Learning Objectives
1.   Learn the technical difference between a conditional and unconditional GAN.
2.   Understand the distinction between the class and noise vector in a conditional GAN.



### Goals
In this notebook, you're going to make a conditional GAN in order to generate hand-written images of digits, conditioned on the digit to be generated (the class vector). This will let you choose what digit you want to generate.

You'll then do some exploration of the generated images to visualize what the noise and class vectors mean.  

### Learning Objectives
1.   Learn the technical difference between a conditional and unconditional GAN.
2.   Understand the distinction between the class and noise vector in a conditional GAN.



In [2]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader

torch.manual_seed(0)

In [3]:
def show_tensor_images(image_tensor,
                       num_images=25,
                       size=(1, 28, 28),
                       nrow=5,
                       show=True):
    """_summary_

    Args:
        image_tensor (_type_): _description_
        num_images (int, optional): _description_. Defaults to 25.
        size (tuple, optional): _description_. Defaults to (1, 28, 28).
        nrow (int, optional): _description_. Defaults to 5.
        show (bool, optional): _description_. Defaults to True.
    """
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=nrow)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    
    if show:
        plt.show()

### Generator and Noise

In [7]:
class Generator(nn.Module):
    def __init__(self, input_dim=10, im_chan=1, hidden_dim=64):
        super(Generator, self).__init__()
        self._input_dim = input_dim
        self._generator = nn.Sequential(
            self._make_gen_block(input_dim, hidden_dim * 4),
            self._make_gen_block(hidden_dim * 4, hidden_dim * 2, kernel_size=4, stride=1),
            self._make_gen_block(hidden_dim * 2, hidden_dim),
            self._make_gen_block(hidden_dim, im_chan, kernel_size=4, final_layer=True)
        )

    def _make_gen_block(self, input_channels, output_channels, kernel_size, stride, final_layer=False):
        if not final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride),
                nn.BatchNorm2d(output_channels),
                nn.ReLU(inplace=True)
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride),
                nn.Tanh()
            )
    
    def call(self, noise):
        x = noise.view(len(noise), self._input_dim, 1, 1)
        return self._generator(x)

def get_noise(n_samples, input_dim, device='cpu'):
    return torch.randn(n_samples, intput_dim, device=device)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, im_chan=1, hidden_dim=64):
        super(Discriminator, self).__init__()
        self._discriminator = nn.Sequential(
            self._make_disc_block(im_chan, hidden_dim),
            self._make_disc_block(hidden_dim, hidden_dim * 2),
            self._make_disc_block(hidden_dim * 2, 1, final_layer=True)
        )

    def _make_disc_block(self, input_channels, output_channels, kernel_size, stride, final_layer=False):
        if not final_layer:
            nn.Sequential(
                nn.Conv2D(input_channels, output_channels, kernel_size, stride),
                nn.BatchNorm2d(output_channels),
                nn.LeakyReLU(on2, inplace=True)
            )
        else:
            return nn.Sequential(
                nn.Conv2d(input_channels, output_channels, kernel_size, stride)
            )

    def forward(self, image):
        disc_pred = self._discriminator(image)
        return disc_pred.view(len(disc_pred), -1)

## Class Input

In conditional GANs, the input vector for the generator will also need to include the class information. The class is represented using a one-hot encoded vector where its length is the number of classes and each index represents a class. The vector is all 0's and a 1 on the chosen class. Given the labels of multiple images (e.g. from a batch) and number of classes, please create one-hot vectors for each label. There is a class within the PyTorch functional library that can help you.

<details>

<summary>
<font size="3" color="green">
<b>Optional hints for <code><font size="4">get_one_hot_labels</font></code></b>
</font>
</summary>

1.   This code can be done in one line.
2.   The documentation for [F.one_hot](https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.one_hot) may be helpful.

</details>


In [12]:
torch.arange(0, 5)
F.one_hot(torch.arange(0, 5), num_classes=5)

tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1]])

In [13]:
import torch.nn.functional as F


def get_one_hot_labels(labels, n_classes):
    return F.one_hot(labels, num_classes=n_classes)

In [14]:
assert (
    get_one_hot_labels(
        labels=torch.Tensor([[0, 2, 1]]).long(),
        n_classes=3
    ).tolist() == 
    [[
      [1, 0, 0], 
      [0, 0, 1], 
      [0, 1, 0]
    ]]
)
# Check that the device of get_one_hot_labels matches the input device
if torch.cuda.is_available():
    assert str(get_one_hot_labels(torch.Tensor([[0]]).long().cuda(), 1).device).startswith("cuda")
    
print("Success!")

Success!


Next, you need to be able to concatenate the one-hot class vector to the noise vector before giving it to the generator. You will also need to do this when adding the class channels to the discriminator.

To do this, you will need to write a function that combines two vectors. Remember that you need to ensure that the vectors are the same type: floats. Again, you can look to the PyTorch library for help.
<details>
<summary>
<font size="3" color="green">
<b>Optional hints for <code><font size="4">combine_vectors</font></code></b>
</font>
</summary>

1.   This code can also be written in one line.
2.   The documentation for [torch.cat](https://pytorch.org/docs/master/generated/torch.cat.html) may be helpful.
3.   Specifically, you might want to look at what the `dim` argument of `torch.cat` does.

</details>


In [21]:
x1 = torch.randn(2, 3)
x2 = torch.randn(2, 3)
x1, x2

(tensor([[ 0.5428, -2.2188,  0.2590],
         [-1.0297, -0.5008,  0.2734]]),
 tensor([[-0.9181, -0.0404,  0.2881],
         [-0.0075, -0.9145, -1.0886]]))

In [22]:
test = torch.cat([x1, x2], axis=1)
test

tensor([[ 0.5428, -2.2188,  0.2590, -0.9181, -0.0404,  0.2881],
        [-1.0297, -0.5008,  0.2734, -0.0075, -0.9145, -1.0886]])

In [23]:
test2 = torch.cat([x1, x2], axis=0)
test2

tensor([[ 0.5428, -2.2188,  0.2590],
        [-1.0297, -0.5008,  0.2734],
        [-0.9181, -0.0404,  0.2881],
        [-0.0075, -0.9145, -1.0886]])

In [29]:
def combine_vectors(x, y):
    return torch.cat([x.float(), y.float()], axis=1)

In [30]:
combined = combine_vectors(torch.tensor([[1, 2], [3, 4]]), torch.tensor([[5, 6], [7, 8]]))
if torch.cuda.is_available():
    # Check that it doesn't break with cuda
    cuda_check = combine_vectors(torch.tensor([[1, 2], [3, 4]]).cuda(), torch.tensor([[5, 6], [7, 8]]).cuda())
    assert str(cuda_check.device).startswith("cuda")
# Check exact order of elements
assert torch.all(combined == torch.tensor([[1, 2, 5, 6], [3, 4, 7, 8]]))
# Tests that items are of float type
assert (type(combined[0][0].item()) == float)
# Check shapes
combined = combine_vectors(torch.randn(1, 4, 5), torch.randn(1, 8, 5));
assert tuple(combined.shape) == (1, 12, 5)
assert tuple(combine_vectors(torch.randn(1, 10, 12).long(), torch.randn(1, 20, 12).long()).shape) == (1, 30, 12)
# Check that the float transformation doesn't happen after the inputs are concatenated
assert tuple(combine_vectors(torch.randn(1, 10, 12).long(), torch.randn(1, 20, 12)).shape) == (1, 30, 12)
print("Success!")

Success!


## Training
Now you can start to put it all together!
First, you will define some new parameters:

*   mnist_shape: the number of pixels in each MNIST image, which has dimensions 28 x 28 and one channel (because it's black-and-white) so 1 x 28 x 28
*   n_classes: the number of classes in MNIST (10, since there are the digits from 0 to 9)

import